In [36]:
import numpy as np

class NaiveBayes:
  # all probabilities inside the class are log

  def __init__(self, *, param_func, prob_func):
    # user-provided function to compute the distribution parameters
    # the function should return an list of parameter arrays (...)
    self.param_func = param_func

    # user-provided function to compute the probabilities from the parameters
    # the input are the parameter arrays and the X
    self.prob_func = prob_func

  def fit(self, X, y):
    # retrieve the number of samples and features from X
    self.n_samples, self.n_features = X.shape

    # collect the individual classes, and record their counts to compute priors
    self.classes, counts = np.unique(y, return_counts=True)

    # compute the parameters for every class - we transpose the array so that
    # the final dimensions are (parameters, classes, features)
    self.params = np.array(
      [self.param_func(X[c==y]) for c in self.classes]
    ).transpose(1, 0, 2)

    # compute the priors from the counts
    self.priors = np.log(counts/self.n_samples)

    return self

  def posteriors(self, X):
    # reshape X to fit the array dimensions (samples, classes, features)
    X = np.reshape(X, (-1, 1, self.n_features))

    # compute the probabilities of the samples (...)
    probs = np.log(self.prob_func(*self.params[:,np.newaxis], X))
    return probs.sum(axis=2) + self.priors[np.newaxis]

  def predict_proba(self, X):
    exp_post = np.exp(self.posteriors(X))
    return exp_post / exp_post.sum(axis=1)[:,np.newaxis]

  def predict(self, X):
    return self.classes[np.argmax(self.posteriors(X), axis=1)]

### Gaussian Naive Bayes

In [40]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

def mean_var(X):
  return [X.mean(axis=0), X.var(axis=0)]
def gauss(mean, var, x):
  return np.exp(-(x-mean)**2/(2*var)) / np.sqrt(var*2*np.pi)

nb = NaiveBayes(param_func=mean_var, prob_func=gauss).fit(X_train, y_train)
y_pred = nb.predict(X_test)
(y_pred == y_test).mean()

0.9666666666666667

### Bernoulli Naive Bayes

In [29]:
X_S = [[0,1,1,1,0,0],
    [0,0,1,1,1,0],
    [1,1,0,0,0,0],
    [1,1,0,0,0,1],
    [1,0,1,0,1,0]]
X_E = [[1,1,1,1,1,1,1],
  [0,1,1,1,1,0,0],
  [0,0,1,0,0,1,1],
  [1,0,1,1,1,1,0],
  [1,1,0,0,1,0,0]]

X = np.c_[X_S, X_E].T
y = len(X_S[0])*[0] + len(X_E[0])*[1]

nb = NaiveBayes(
  param_func=lambda X: [X.mean(axis=0)],
  prob_func=lambda theta, x: theta*x + (1-theta)*(1-x)
).fit(X, y)

nb.predict([1,0,1,1,0])

array([1])

In [30]:
nb.predict_proba([1,0,1,1,0])

array([[0.19237241, 0.80762759]])